In [1]:
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
from tasho.OCPGenerator import OCPGenerator
from robotshyu import Robot as rob
from time import sleep
import numpy as np
import casadi as cs
import pybullet as p
import tasho
import robotshyu
import os

pybullet build time: May 20 2022 19:44:17


# P2P Redundant manipulator

In [2]:
robot = rob.Robot("kinova")
link_name = 7
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 1, 0, 0.6], [1, 0, 0, 0.0], [0, 0, -1, 0.25], [0, 0, 0, 1]]
    ))
q0 = [1.0193752249977548, -0.05311582280659044, -2.815452580946695,
    1.3191046402052224, 2.8582660722530533, 1.3988994390898029, 1.8226311094569714]
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 7-DoF robot: kinova


In [3]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [4]:
task_P2P.write_task_graph("P2P_Task.svg")

In [5]:
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1),
                              Regularization(task_P2P.variables['qdd_'+robot.name], 0.1))

In [6]:
task_P2P.write_task_graph("P2P_Task2.svg")

In [7]:
horizon_steps = 10
horizon_period = 3.0
OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      529
Number of nonzeros in inequality constraint Jacobian.:      217
Number of nonzeros in Lagrangian Hessian.............:      168

Total number of variables............................:      224
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      164
Total number of inequality constrai

In [8]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

from tasho import WorldSimulator
import pybullet as p

obj = WorldSimulator.WorldSimulator()

    # Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
joint_indices = [0, 1, 2, 3, 4, 5, 6]

obj.resetJointState(robotID, joint_indices, q0)

for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetJointState(
        robotID, joint_indices, qsol[i]
    )

sleep(0.5)
obj.end_simulation()

AttributeError: 'Robot' object has no attribute 'urdf_path'

# Object picking MPC

In [13]:
robot = rob.Robot("kinova")
link_name = 7

# Define initial conditions of the robot
q0_val = [0, -0.523598, 0, 2.51799, 0, -0.523598, -1.5708]
qd0_val = [0] * robot.ndof
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, 7, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("P2P_Task4.svg")

Loading robot params from json ...
Loaded 7-DoF robot: kinova


In [14]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_kinova'])
task_P2P.remove_terminal_constraints('rot_con_pose_7_kinova_vs_goal',
                                    'trans_con_pose_7_kinova_vs_goal')
task_P2P.write_task_graph("P2P_Task5.svg")

In [15]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_kinova']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_7_kinova_vs_goal'], 1.0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_7_kinova_vs_goal'], 1), 
                            Regularization(task_P2P.variables['qd_kinova'], 1e-4),
                            Regularization(task_P2P.variables['qdd_kinova'], 1e-3))

task_P2P.write_task_graph("P2P_Task6.svg")

In [16]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_kinova'].x
qd = pOCP.stage_tasks[0].variables['qd_kinova'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_kinova'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_kinova'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_kinova'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_kinova", "var":"qd_kinova", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_7_kinova_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,846]:

b3Printf: /home/mtplnr/mpc_ws/postech_lec/.venv/lib/python3.8/site-packages/robotshy

sh: 1: export: : bad variable name


hread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [17]:
dir_casadi_func = "casadi_dir"
os.makedirs("./"+dir_casadi_func, exist_ok=True)
vars_db = tc.generate_MPC_component("./"+dir_casadi_func+"/", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [18]:
MPC_component = MPC("kinova_obj_pickup", "./"+dir_casadi_func+"/"+ tc.name + ".json")

In [19]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
kinovaID = obj.add_robot(position, orientation, "kinova")

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5, 6]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")
obj.resetJointState(kinovaID, joint_indices, q0_val)
obj.setController(
    kinovaID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

In [ ]:
# Execute the MPC loop
q_log = []
q_dot_log = []
predicted_pos_log = []

for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(kinovaID, joint_indices)
    qd_now = obj.readJointVelocities(kinovaID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_kinova"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_kinova"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 1, 0, predicted_pos[0]]),
        cs.hcat([1, 0, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_kinova"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_kinova"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_kinova"]["val"] * t_mpc).full()
    obj.setController(
        kinovaID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.15777500346789, 0.41]
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.

----------- MPC execution -----------
Predicted position of cube [0.49999968066805, 0.31677326454043, 0.27249110475156]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 268.00us ( 67.00us) 268.89us ( 67.22us)         4
       nlp_g  |  64.00us ( 16.00us)  63.25us ( 15.81us)         4
    nlp_grad  | 219.00us (219.00us) 219.02us (219.02us)         1
  nlp_grad_f  | 711.00us (177.75us) 713.82us (178.45us)         4
  nlp_hess_l  |   7.89ms (  2.63ms)   3.10ms (  1.03ms)         3
   nlp_jac_g  |  86.00us ( 21.50us)  85.82us ( 21.45us)         4
       total  |  19.66ms ( 19.66ms)   6.85ms (  6.85ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999968506725, 0.32652397891008, 0.27249052544209]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 320.00us ( 80.00us) 320.47us ( 80.12us)         4
       nlp_g  |  73.00us ( 18.25us)  73.14us ( 18.29us)         4
    nlp_grad  |   3.74ms (  3.74ms

----------- MPC execution -----------
Predicted position of cube [0.49999522971114, 0.43300391338629, 0.084989917541916]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 223.00us ( 55.75us) 194.87us ( 48.72us)         4
       nlp_g  |  46.00us ( 11.50us)  44.69us ( 11.17us)         4
    nlp_grad  | 166.00us (166.00us) 165.88us (165.88us)         1
  nlp_grad_f  |   2.07ms (517.50us) 538.54us (134.63us)         4
  nlp_hess_l  |  12.87ms (  4.29ms)   2.32ms (774.00us)         3
   nlp_jac_g  | 167.00us ( 41.75us)  63.49us ( 15.87us)         4
       total  |  19.29ms ( 19.29ms)   5.12ms (  5.12ms)         1
----------- MPC execution -----------
Predicted position of cube [0.4999950969829, 0.43465349718153, 0.084989958498636]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 756.00us (151.20us) 236.35us ( 47.27us)         5
       nlp_g  |  53.00us ( 10.60us)  52.32us ( 10.46us)         5
    nlp_grad  | 157.00us (157.00u

----------- MPC execution -----------
Predicted position of cube [0.49999354750716, 0.43634547411444, 0.084989987246302]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.91ms (635.33us) 191.10us ( 63.70us)         3
       nlp_g  |  56.00us ( 18.67us)  55.13us ( 18.38us)         3
    nlp_grad  | 367.00us (367.00us) 220.54us (220.54us)         1
  nlp_grad_f  |   1.39ms (464.67us) 531.71us (177.24us)         3
  nlp_hess_l  |  13.56ms (  6.78ms)   2.27ms (  1.14ms)         2
   nlp_jac_g  | 198.00us ( 66.00us)  68.45us ( 22.82us)         3
       total  |  68.86ms ( 68.86ms)   5.70ms (  5.70ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354691867, 0.43634546654046, 0.084989995310537]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 231.00us ( 77.00us) 231.28us ( 77.09us)         3
       nlp_g  | 145.00us ( 48.33us)  63.55us ( 21.18us)         3
    nlp_grad  | 264.00us (264.00

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214853, 0.084989999999964]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 517.00us (172.33us) 155.23us ( 51.74us)         3
       nlp_g  |  37.00us ( 12.33us)  36.23us ( 12.08us)         3
    nlp_grad  | 164.00us (164.00us) 159.32us (159.32us)         1
  nlp_grad_f  |   1.67ms (558.00us) 398.13us (132.71us)         3
  nlp_hess_l  |   2.06ms (  1.03ms)   1.61ms (804.41us)         2
   nlp_jac_g  |  47.00us ( 15.67us)  47.31us ( 15.77us)         3
       total  |  14.70ms ( 14.70ms)   4.28ms (  4.28ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214854, 0.084989999999972]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   8.14ms (  2.71ms) 142.27us ( 47.42us)         3
       nlp_g  |  38.00us ( 12.67us)  36.91us ( 12.30us)         3
    nlp_grad  | 156.00us (156.00

...

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 518.00us (172.67us) 216.81us ( 72.27us)         3
       nlp_g  | 338.00us (112.67us)  59.88us ( 19.96us)         3
    nlp_grad  | 253.00us (253.00us) 252.73us (252.73us)         1
  nlp_grad_f  |   2.59ms (863.33us) 598.66us (199.55us)         3
  nlp_hess_l  |   4.36ms (  2.18ms)   2.32ms (  1.16ms)         2
   nlp_jac_g  | 414.00us (138.00us)  75.91us ( 25.30us)         3
       total  |  18.99ms ( 18.99ms)   5.72ms (  5.72ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 165.00us ( 55.00us) 165.32us ( 55.11us)         3
       nlp_g  |  48.00us ( 16.00us)  47.08us ( 15.69us)         3
    nlp_grad  | 159.00us (159.00us) 159.57us (159.57

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   3.33ms (  1.11ms) 165.01us ( 55.00us)         3
       nlp_g  |  44.00us ( 14.67us)  44.27us ( 14.76us)         3
    nlp_grad  | 208.00us (208.00us) 203.34us (203.34us)         1
  nlp_grad_f  |   3.02ms (  1.01ms) 385.21us (128.40us)         3
  nlp_hess_l  |   6.28ms (  3.14ms)   1.60ms (800.05us)         2
   nlp_jac_g  | 130.00us ( 43.33us)  52.44us ( 17.48us)         3
       total  |  40.63ms ( 40.63ms)   4.36ms (  4.36ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 140.00us ( 46.67us) 140.46us ( 46.82us)         3
       nlp_g  |  37.00us ( 12.33us)  38.10us ( 12.70us)         3
    nlp_grad  | 164.00us (164.00us) 164.52us (164.52

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 134.00us ( 44.67us) 134.55us ( 44.85us)         3
       nlp_g  |  38.00us ( 12.67us)  38.14us ( 12.71us)         3
    nlp_grad  | 154.00us (154.00us) 154.81us (154.81us)         1
  nlp_grad_f  | 368.00us (122.67us) 368.50us (122.83us)         3
  nlp_hess_l  |   9.49ms (  4.75ms)   1.51ms (752.71us)         2
   nlp_jac_g  |  45.00us ( 15.00us)  45.79us ( 15.26us)         3
       total  |  11.67ms ( 11.67ms)   3.68ms (  3.68ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.25ms (417.67us) 158.91us ( 52.97us)         3
       nlp_g  |   1.46ms (486.33us)  41.83us ( 13.94us)         3
    nlp_grad  | 154.00us (154.00us) 153.99us (153.99

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.55ms (515.67us) 143.91us ( 47.97us)         3
       nlp_g  | 134.00us ( 44.67us)  39.89us ( 13.30us)         3
    nlp_grad  | 159.00us (159.00us) 159.66us (159.66us)         1
  nlp_grad_f  |   1.25ms (416.00us) 403.93us (134.64us)         3
  nlp_hess_l  |  10.28ms (  5.14ms)   1.55ms (775.49us)         2
   nlp_jac_g  |  47.00us ( 15.67us)  47.77us ( 15.92us)         3
       total  |  29.34ms ( 29.34ms)   3.89ms (  3.89ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.65ms (551.33us) 138.96us ( 46.32us)         3
       nlp_g  |  36.00us ( 12.00us)  36.91us ( 12.30us)         3
    nlp_grad  |   1.89ms (  1.89ms) 160.36us (160.36

----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   3.63ms (  1.21ms) 150.79us ( 50.26us)         3
       nlp_g  |  43.00us ( 14.33us)  41.61us ( 13.87us)         3
    nlp_grad  | 183.00us (183.00us) 184.18us (184.18us)         1
  nlp_grad_f  | 399.00us (133.00us) 399.50us (133.17us)         3
  nlp_hess_l  |   7.75ms (  3.88ms)   1.61ms (803.36us)         2
   nlp_jac_g  |  48.00us ( 16.00us)  48.07us ( 16.02us)         3
       total  |  42.75ms ( 42.75ms)   4.27ms (  4.27ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999354662903, 0.43634546214857, 0.08499]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 306.00us (102.00us) 139.48us ( 46.49us)         3
       nlp_g  |  89.00us ( 29.67us)  32.72us ( 10.91us)         3
    nlp_grad  | 155.00us (155.00us) 155.50us (155.50

In [ ]:
obj.end_simulation()

max(MPC_component.solver_times)